<a href="https://www.kaggle.com/code/haddonalexander25/space-titanic-with-decision-tree?scriptVersionId=96960585" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
open_file = open("../input/spaceship-titanic/train.csv")

train_df = pd.read_csv(open_file)

train_df.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
train_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [4]:
columns_to_drop = [] #I'm going to keep a list of the items that end up hot encoded

train_df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [5]:
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


There are a number of null values within most of the categories. I'm going to work on cleaning these up before I start looking at the model. 

In [6]:
#starting with HomePlanet 

train_df["HomePlanet"].value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [7]:
train_df["HomePlanet"].fillna("Unknown Planet", inplace = True)

In [8]:
planet_dummies = pd.get_dummies(train_df["HomePlanet"])

columns_to_drop.append("HomePlanet")

planet_dummies

,Earth,Europa,Mars,Unknown Planet
0,0,1,0,0
1,1,0,0,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
8688,0,1,0,0
8689,1,0,0,0
8690,1,0,0,0
8691,0,1,0,0


In [9]:
train_df = pd.concat([train_df, planet_dummies], axis = 1)

In [10]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Earth,Europa,Mars,Unknown Planet
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0


In [11]:
train_df["CryoSleep"].isnull().sum()

217

In [12]:
train_df["CryoSleep"].value_counts()

False    5439
True     3037
Name: CryoSleep, dtype: int64

In [13]:
(217)/(5439 + 3037)

0.02560169891458235

In [14]:
train_df["CryoSleep"].fillna(False, inplace = True) #since the missing values are only 2.56% of the cyrosleep column, I am going to just replace them with False. The primary value in that column

In [15]:
train_df["CryoSleep"].replace([True, False], [1, 0], inplace = True)

Now onto the Cabin. Reading the backup for this dataset, it appears the meaninful part of this column will likely be what deck the passenger was on, and if they were starbord or port. 

In [16]:
train_df["Cabin"].value_counts()

G/734/S     8
G/109/P     7
B/201/P     7
G/1368/P    7
G/981/S     7
           ..
G/556/P     1
E/231/S     1
G/545/S     1
G/543/S     1
F/947/P     1
Name: Cabin, Length: 6560, dtype: int64

In [17]:
#I want to seperate the individual decks the passengers are on 

train_df["Deck"] = train_df["Cabin"].astype(str).str[0]

In [18]:
train_df["Deck"]

0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: Deck, Length: 8693, dtype: object

In [19]:
train_df["Deck"].value_counts()

F    2794
G    2559
E     876
B     779
C     747
D     478
A     256
n     199
T       5
Name: Deck, dtype: int64

In [20]:
#the percentage of the null values to values of the most frequent 
#value in the column are higher than 
#the CryoSleep column, therefore I am going to opt to make and "Unknown Deck" column
#via hot encoding

train_df["Deck"].replace("n", "Deck Unknown", inplace = True)

columns_to_drop.append("Deck")

deck_dummies = pd.get_dummies(train_df["Deck"])

train_df = pd.concat([train_df, deck_dummies], axis = 1)

In [21]:
train_df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Earth', 'Europa', 'Mars', 'Unknown Planet',
       'Deck', 'A', 'B', 'C', 'D', 'Deck Unknown', 'E', 'F', 'G', 'T'],
      dtype='object')

In [22]:
train_df["Side"] = train_df["Cabin"].astype(str).str[-1]

In [23]:
train_df["Side"].value_counts()

S    4288
P    4206
n     199
Name: Side, dtype: int64

In [24]:
train_df["Side"].replace("n", "Side Unknown", inplace = True)

In [25]:
side_dummies = pd.get_dummies(train_df['Side'])

columns_to_drop.append("Side")

side_dummies

,P,S,Side Unknown
0,1,0,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
8688,1,0,0
8689,0,1,0
8690,0,1,0
8691,0,1,0


In [26]:
train_df = pd.concat([train_df, side_dummies],axis = 1)

In [27]:
train_df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Earth', 'Europa', 'Mars', 'Unknown Planet',
       'Deck', 'A', 'B', 'C', 'D', 'Deck Unknown', 'E', 'F', 'G', 'T', 'Side',
       'P', 'S', 'Side Unknown'],
      dtype='object')

In [28]:
print(train_df["Destination"].isnull().sum())

train_df["Destination"].value_counts()

182


TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64

In [29]:
train_df["Destination"].fillna("TRAPPIST-1e", inplace = True)

In [30]:
dest_dummies = pd.get_dummies(train_df["Destination"])

columns_to_drop.append("Destination")

dest_dummies

,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
8688,1,0,0
8689,0,1,0
8690,0,0,1
8691,1,0,0


In [31]:
train_df = pd.concat([train_df, dest_dummies], axis = 1)

In [32]:
vip_true = len(train_df[train_df["VIP"] == True])

vip_false = len(train_df[train_df["VIP"] == False])

vip_null = train_df["VIP"].isnull().sum()

print(vip_true)

print(vip_false)

print(vip_null)

print("null/false: " + str(vip_null/vip_false))

199
8291
203
null/false: 0.024484380653720904


In [33]:
train_df["VIP"].fillna(False, inplace = True)

train_df["VIP"].replace([True, False], [1, 0], inplace = True)

In [34]:
train_df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Earth', 'Europa', 'Mars', 'Unknown Planet',
       'Deck', 'A', 'B', 'C', 'D', 'Deck Unknown', 'E', 'F', 'G', 'T', 'Side',
       'P', 'S', 'Side Unknown', '55 Cancri e', 'PSO J318.5-22',
       'TRAPPIST-1e'],
      dtype='object')

In [35]:
age_med = train_df["Age"].median()

print(age_med)

train_df["Age"].fillna(age_med, inplace = True)

train_df["Age"].isnull().sum()

27.0


0

In [36]:
train_df.isnull().sum()

PassengerId         0
HomePlanet          0
CryoSleep           0
Cabin             199
Destination         0
Age                 0
VIP                 0
RoomService       181
FoodCourt         183
ShoppingMall      208
Spa               183
VRDeck            188
Name              200
Transported         0
Earth               0
Europa              0
Mars                0
Unknown Planet      0
Deck                0
A                   0
B                   0
C                   0
D                   0
Deck Unknown        0
E                   0
F                   0
G                   0
T                   0
Side                0
P                   0
S                   0
Side Unknown        0
55 Cancri e         0
PSO J318.5-22       0
TRAPPIST-1e         0
dtype: int64

In [37]:
train_df["RoomService"].describe()

count     8512.000000
mean       224.687617
std        666.717663
min          0.000000
25%          0.000000
50%          0.000000
75%         47.000000
max      14327.000000
Name: RoomService, dtype: float64

In [38]:
train_df["RoomService"].fillna(0, inplace = True)

In [39]:
train_df["FoodCourt"].describe()

count     8510.000000
mean       458.077203
std       1611.489240
min          0.000000
25%          0.000000
50%          0.000000
75%         76.000000
max      29813.000000
Name: FoodCourt, dtype: float64

In [40]:
train_df["FoodCourt"].fillna(0, inplace = True)

In [41]:
train_df["Spa"].describe()

count     8510.000000
mean       311.138778
std       1136.705535
min          0.000000
25%          0.000000
50%          0.000000
75%         59.000000
max      22408.000000
Name: Spa, dtype: float64

In [42]:
train_df["Spa"].fillna(0, inplace = True)

In [43]:
train_df["VRDeck"].describe()

count     8505.000000
mean       304.854791
std       1145.717189
min          0.000000
25%          0.000000
50%          0.000000
75%         46.000000
max      24133.000000
Name: VRDeck, dtype: float64

In [44]:
train_df["VRDeck"].fillna(0, inplace = True)

In [45]:
train_df["ShoppingMall"].describe()

count     8485.000000
mean       173.729169
std        604.696458
min          0.000000
25%          0.000000
50%          0.000000
75%         27.000000
max      23492.000000
Name: ShoppingMall, dtype: float64

In [46]:
train_df["ShoppingMall"].fillna(0, inplace = True)

Now I have removed the null values from the data set. It will be important to review these actions after the results of the learning model are produced. 

In [47]:
train_df["Name"].value_counts()

Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64

In [48]:
train_df.isnull().sum()

PassengerId         0
HomePlanet          0
CryoSleep           0
Cabin             199
Destination         0
Age                 0
VIP                 0
RoomService         0
FoodCourt           0
ShoppingMall        0
Spa                 0
VRDeck              0
Name              200
Transported         0
Earth               0
Europa              0
Mars                0
Unknown Planet      0
Deck                0
A                   0
B                   0
C                   0
D                   0
Deck Unknown        0
E                   0
F                   0
G                   0
T                   0
Side                0
P                   0
S                   0
Side Unknown        0
55 Cancri e         0
PSO J318.5-22       0
TRAPPIST-1e         0
dtype: int64

In [49]:
columns_to_drop.append("Name")
columns_to_drop.append("PassengerId")
columns_to_drop.append("Cabin")

train_df.drop(columns_to_drop, axis = 1, inplace = True)

In [50]:
train_df.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Transported', 'Earth', 'Europa', 'Mars',
       'Unknown Planet', 'A', 'B', 'C', 'D', 'Deck Unknown', 'E', 'F', 'G',
       'T', 'P', 'S', 'Side Unknown', '55 Cancri e', 'PSO J318.5-22',
       'TRAPPIST-1e'],
      dtype='object')

In [51]:
train_df.corr()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Earth,...,E,F,G,T,P,S,Side Unknown,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
CryoSleep,1.000000,-0.071323,-0.078281,-0.244089,-0.205928,-0.207798,-0.198307,-0.192721,0.460132,-0.109133,...,-0.110644,-0.220150,0.241399,-0.017579,-0.023858,0.016861,0.023356,0.065589,0.087764,-0.113380
Age,-0.071323,1.000000,0.091863,0.068629,0.127390,0.033148,0.120946,0.099590,-0.074233,-0.200003,...,0.028212,-0.018397,-0.209928,0.013734,-0.011621,0.010609,0.003365,0.016817,-0.028121,0.002832
VIP,-0.078281,0.091863,1.000000,0.056566,0.125499,0.018412,0.060991,0.123061,-0.037261,-0.162341,...,-0.012912,-0.057578,-0.098863,-0.003672,0.008798,-0.011017,0.007429,0.045167,-0.000592,-0.039617
RoomService,-0.244089,0.068629,0.056566,1.000000,-0.015126,0.052337,0.009244,-0.018624,-0.241124,-0.138316,...,0.043935,0.085627,-0.145289,0.007525,0.006991,-0.014918,0.026508,-0.023441,-0.063354,0.060680
FoodCourt,-0.205928,0.127390,0.125499,-0.015126,1.000000,-0.013717,0.221468,0.224572,0.045583,-0.208772,...,-0.038889,-0.131988,-0.149911,0.014267,-0.019682,0.019625,0.000160,0.134952,-0.065400,-0.078269
ShoppingMall,-0.207798,0.033148,0.018412,0.052337,-0.013717,1.000000,0.014542,-0.007849,0.009391,-0.069099,...,0.031747,0.074473,-0.108538,-0.006787,0.020940,-0.018018,-0.009738,-0.013739,-0.031245,0.031855
Spa,-0.198307,0.120946,0.060991,0.009244,0.221468,0.014542,1.000000,0.147658,-0.218545,-0.155374,...,-0.011736,-0.087538,-0.124342,0.036316,-0.005700,-0.002116,0.026119,0.075096,-0.053303,-0.032897
VRDeck,-0.192721,0.099590,0.123061,-0.018624,0.224572,-0.007849,0.147658,1.000000,-0.204874,-0.152875,...,-0.017905,-0.094548,-0.127434,0.012369,0.009089,-0.006950,-0.007138,0.089667,-0.044861,-0.051119
Transported,0.460132,-0.074233,-0.037261,-0.241124,0.045583,0.009391,-0.218545,-0.204874,1.000000,-0.169019,...,-0.097965,-0.087753,0.016269,-0.014568,-0.101397,0.101455,-0.000340,0.108722,0.000092,-0.096319
Earth,-0.109133,-0.200003,-0.162341,-0.138316,-0.208772,-0.069099,-0.155374,-0.152875,-0.169019,1.000000,...,-0.052634,0.066562,0.578136,-0.025444,0.020006,-0.015226,-0.015948,-0.149534,0.232218,-0.013949


In [52]:
train_df["CryoSleep"].value_counts()

0    5656
1    3037
Name: CryoSleep, dtype: int64

In [53]:
y = train_df["Transported"]

train_df.drop("Transported", axis = 1, inplace = True)

In [54]:
from sklearn import tree

X = train_df

model = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = 8)

model = model.fit(X, y)

print(model.get_depth())

print(model.get_n_leaves())

8
151


Now that we have trained our model, we need to test it. However, it will be necessary to transform the test data so that it reflects all the changes that the training data had. 

In [55]:
open_file = open("../input/spaceship-titanic/test.csv")

test_df = pd.read_csv(open_file)

test_df_copy = test_df

In [56]:
len(test_df)

4277

In [57]:
#planets 
test_df["HomePlanet"] = test_df["HomePlanet"].fillna("Unknown Planet")
planet_dummies = pd.get_dummies(test_df["HomePlanet"])
test_df = pd.concat([test_df, planet_dummies], axis = 1)

#CryoSleep
test_df["CryoSleep"] = test_df["CryoSleep"].fillna(False)
test_df["CryoSleep"] = test_df["CryoSleep"].replace([True, False], [1, 0])

#Deck
test_df["Deck"] = test_df["Cabin"].astype(str).str[0]
test_df["Deck"] = test_df["Deck"].replace("n", "Deck Unknown")
deck_dummies = pd.get_dummies(test_df["Deck"])
test_df = pd.concat([test_df, deck_dummies], axis = 1)

#Side
test_df["Side"] = test_df["Cabin"].astype(str).str[-1]
test_df["Side"] = test_df["Side"].replace("n", "Side Unknown")
side_dummies = pd.get_dummies(test_df['Side'])
test_df = pd.concat([test_df, side_dummies],axis = 1)

#Destination
test_df["Destination"] = test_df["Destination"].fillna("TRAPPIST-1e")
dest_dummies = pd.get_dummies(test_df["Destination"])
test_df = pd.concat([test_df, dest_dummies], axis = 1)

#VIP
test_df["VIP"] = test_df["VIP"].fillna(False)
test_df["VIP"] = test_df["VIP"].replace([True, False], [1, 0])


#Age
age_med = test_df["Age"].median()
test_df["Age"] = test_df["Age"].fillna(age_med)

#FoodCourt
test_df["FoodCourt"] = test_df["FoodCourt"].fillna(0)

#RoomService
test_df["RoomService"] = test_df["RoomService"].fillna(0)

#Spa
test_df["Spa"] = test_df["Spa"].fillna(0)

#VRDeck
test_df["VRDeck"] = test_df["VRDeck"].fillna(0)

#ShoppingMall
test_df["ShoppingMall"] = test_df["ShoppingMall"].fillna(0)

In [58]:
test_df.isnull().sum()

PassengerId         0
HomePlanet          0
CryoSleep           0
Cabin             100
Destination         0
Age                 0
VIP                 0
RoomService         0
FoodCourt           0
ShoppingMall        0
Spa                 0
VRDeck              0
Name               94
Earth               0
Europa              0
Mars                0
Unknown Planet      0
Deck                0
A                   0
B                   0
C                   0
D                   0
Deck Unknown        0
E                   0
F                   0
G                   0
T                   0
Side                0
P                   0
S                   0
Side Unknown        0
55 Cancri e         0
PSO J318.5-22       0
TRAPPIST-1e         0
dtype: int64

In [59]:
print(columns_to_drop)

['HomePlanet', 'Deck', 'Side', 'Destination', 'Name', 'PassengerId', 'Cabin']


In [60]:
test_df = test_df.drop(columns_to_drop, axis = 1)

In [61]:
test_df.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Earth', 'Europa', 'Mars', 'Unknown Planet', 'A', 'B',
       'C', 'D', 'Deck Unknown', 'E', 'F', 'G', 'T', 'P', 'S', 'Side Unknown',
       '55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e'],
      dtype='object')

In [62]:
X_test = test_df

predictions = model.predict(X_test)


In [63]:
my_submission = pd.DataFrame({'PassengerId': test_df_copy["PassengerId"], "Transported": predictions})

my_submission = my_submission.set_index("PassengerId")

In [64]:
my_submission

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [65]:
my_submission.to_csv("submission.csv")